# In the Name of God
Name : Mohammad Mohajel Sadegi

SID  : 810199483


## Abstract:
Implementation of BFS, IDS and A* search 

## Importing libraries:

In [48]:
import copy
from collections import Counter
from queue import Queue
import time

## Class state

In [49]:
class State:
    def __init__(self, rocky_nodes) -> None:
        self.position = 0
        self.recipes_seen = [] #be very carefull changed
        self.morids_seen = []
        self.prev_state = None
        self.rocky_nodes = dict.fromkeys(rocky_nodes, 0)
        self.rocky_remain = 0
        self.cost = 0

    def __members(self):
        return (self.position, self.recipes_seen, self.morids_seen, self.rocky_remain)


    def __eq__(self, other):
        if type(other) is type(self):
            return self.__members() == other.__members()
        return False

    def __hash__(self):
        return hash(f"{self.position},{self.recipes_seen},{self.morids_seen},{self.rocky_remain}")
           

## Class graph:

In [50]:
class Graph:
    def __init__(self, v :int) -> None:
        self.v = v
        self.edges = [[] for _ in range(v)]
        self.morids = None #make it dictionary
        self.recipes = []
        self.start_position = 0
        self.rocky_nodes = []

    def add_edge(self, u :int, v :int) -> None:
        self.edges[u].append(v)
        self.edges[v].append(u)

    def is_rocky(self, node :int) -> bool:
        return (node in self.rocky_nodes)

    def is_recipes(self, node :int) -> bool:
        return (node in self.recipes)

    def is_morid(self, node :int) -> bool:
        return (node in list(self.morids.keys()))

    def get_neighbors(self, s :State):
        return self.edges[s.position]


## Read input and create graph class

In [51]:
def create_graph(path :str) -> Graph:
    file = open(path, "r")
    n, m = map(int, file.readline().split(" "))
    g = Graph(n)

    for _ in range(m):
        u, v = map(int, file.readline().split(" "))
        g.add_edge(u - 1, v - 1)

    h = int(file.readline())
    rocky_nodes = list(map(int, file.readline().split(" ")))
    rocky_nodes = [x - 1 for x in rocky_nodes]
    g.rocky_nodes = rocky_nodes
    s = int(file.readline())

    morids = {}
    for _ in range(s):
        line = list(map(int, file.readline().split(" ")))
        line = [x - 1 for x in line]
        morids.update({line[0] : line[2:]})
    g.morids = morids
    
    g.start_position = (int(file.readline()) - 1)

    recipes = []
    for recipe in morids.values():
        recipes.extend(recipe)

    recipes = list(dict.fromkeys(recipes)) # Remove Duplicates From a Python List 
    g.recipes = recipes
    return g

g1 = create_graph("input.txt")
g2 = create_graph("input2.txt")
g3 = create_graph("input3.txt")
g4 = create_graph("input4.txt")


## Create initial state

In [52]:
def create_initial_state(g :Graph) -> State:
    s = State(g.rocky_nodes)
    s.position = g.start_position
    return s

initial_state1 = create_initial_state(g1)
initial_state2 = create_initial_state(g2)
initial_state3 = create_initial_state(g3)
initial_state4 = create_initial_state(g4)

#BUG
ss1 = create_initial_state(g3)
ss2 = create_initial_state(g3)
ss1.cost = 10
hh = {ss1, initial_state2}
if ss2 in hh:
    print("yes!")

# if ss1 == ss2 :
#     print("ok")


# a = {1,2,3}
# f = {3,1,2}

# x = [a , {1}, "tt" ]
# if f in x :
#     print("ok")
 

yes!


## Transition module and goal state implementation:

In [53]:
counter = 0 # remember to initilize it to 0 in begenning

def reached_goal(s :State, g :Graph) -> bool:
    return len(g.morids) == len(s.morids_seen)

def transition(s :State, g :Graph, p :int) -> State:
    global counter
    counter += 1

    # if s.prev_state == None: #initial_state
    #     print(f"start : {s.position}\n")
    # else:
    #     print(f"{s.prev_state.position} => {s.position} \n")

    next_state = copy.deepcopy(s)
    next_state.prev_state = s
    next_state.cost += 1
    if s.rocky_remain != 0:
        next_state.rocky_remain -= 1
    else:
        next_state.position = p
        if g.is_rocky(p):
            next_state.rocky_remain = next_state.rocky_nodes[p]
            next_state.rocky_nodes[p] += 1

        elif(g.is_recipes(p) and p not in s.recipes_seen):
            next_state.recipes_seen.append(p)

        elif(g.is_morid(p) and p not in s.morids_seen and Counter(g.morids[p]) == Counter(s.recipes_seen)):
            next_state.morids_seen.append(p)
            
    # print(f"going to node {next_state.position + 1}, cost:{next_state.cost}")
    return next_state
    

## Print_path function:
to print path after finding it!

In [54]:
def print_path(final_state : State) -> None:
    print("PATH::")
    s = final_state
    while True:
        print(f"{s.position + 1} <== ", end = " ")
        s = s.prev_state
        if s.prev_state == None: #s is first node
            print(f"{s.position + 1} ")
            break
         

## Test function:

In [55]:
def test(g :Graph, method) -> None:
    global counter
    counter = 0
    start = time.time()
    path = method(g, create_initial_state(g))
    end = time.time()
    print(f"time:{end - start}")
    print_path(path)
    print(f"cost:{path.cost},")
    print(f"states:{counter},")

## BFS implementation:

### With QUEUE:

In [56]:
def bfs(g :Graph, initial_state :State) -> State:
    q = Queue()
    q.put(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:
        state = q.get()
        #for test
        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")

        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.put(t)
                explored.add(t)
                # print(f"{t.prev_state.position + 1} => {t.position + 1}")
        # print("\n")

### With List:

In [57]:
def bfs2(g :Graph, initial_state :State) -> State:
    q = []
    q.append(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:
        state = q.pop(0)
        #for test
        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")

        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.append(t)
                explored.add(t)
                # print(f"{t.prev_state.position + 1} => {t.position + 1}")
        # print("\n")

### With List comprehension:

In [58]:
def bfs3(g :Graph, initial_state :State) -> State:
    q = [initial_state]
    # q.append(initial_state)
    explored = [initial_state,]
    cost = 0
    while 1:
        state = q.pop(0)
        if reached_goal(state, g):
            return state

        #for test
        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")
        
        states = [transition(state, g, x) for x in g.get_neighbors(state)]
        new_states = [x for x in states if x not in explored]
        q.extend(new_states)
        explored.extend(new_states)

### A little greedy:

In [59]:
def h(s :State):
    return len(s.recipes_seen) + len(s.morids_seen)

def bfs4(g :Graph, initial_state :State) -> State:
    max_h = 0
    q = Queue()
    q.put(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:

        while 1:
            state = q.get()

            if h(state) > max_h:
                max_h = h(state)
                break
            elif h(state) >= (max_h - 1):
                break

        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")
            
        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.put(t)
                explored.add(t)

## BFS Test:

### Test1:

In [60]:
test(g1, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
time:0.006826877593994141
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:84,


PATH:

    8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1

|  | MinCost | StatesWatched | MeanTime |
| --- | --- | --- | --- |
| BFS | 8 | 84 | 0.007 |

### Test2:

In [61]:
%%script false --no-raise-error
test(g2, bfs4)

### Test3:

In [62]:
%%script false --no-raise-error
test(g3, bfs)

### Test4:

In [63]:
test(g4, bfs)

cost:1
cost:2
cost:3
cost:4
cost:5
cost:6
cost:7
time:0.0040667057037353516
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:49,


## IDS implementation:

In [64]:
def dfs(state :State, step :int, g :Graph):
    if reached_goal(state, g):
        return state
    if step == 0:
        return None

    for neighbor in g.get_neighbors(state):
        next_state = transition(state, g, neighbor)
        result = dfs(next_state, step - 1, g)
        if result != None:
            return result
    return None

def ids(g:Graph, initial_state :State, h = 0):
    while True:
        h += 1
        print(f"h:{h}")

        result = dfs(initial_state, h, g)
        if result != None:
            return result
        print(f"cant find in h:{h}")

## IDS Tets:

### Test1:

In [65]:
test(g1, ids)

h:1
cant find in h:1
h:2
cant find in h:2
h:3
cant find in h:3
h:4
cant find in h:4
h:5
cant find in h:5
h:6
cant find in h:6
h:7
cant find in h:7
h:8
time:0.3755183219909668
PATH::
8 <==  9 <==  11 <==  10 <==  7 <==  5 <==  4 <==  3 <==  1 
cost:8,
states:4632,


### Test2:

In [66]:
%%script false --no-raise-error
test(g2, ids)

### Test4:

In [67]:
test(g4, ids)

h:1
cant find in h:1
h:2
cant find in h:2
h:3
cant find in h:3
h:4
cant find in h:4
h:5
cant find in h:5
h:6
cant find in h:6
h:7
cant find in h:7
h:8
time:0.05354499816894531
PATH::
5 <==  6 <==  5 <==  4 <==  2 <==  2 <==  3 <==  2 <==  1 
cost:8,
states:580,


## Examples:

In [68]:
x = [ [] for _ in range(5)]
print(x)

[[], [], [], [], []]


In [69]:
a = [1, 2, 3]
b = [4 ,5, 6]
a.extend(b)
print(a)

[1, 2, 3, 4, 5, 6]


In [70]:
# #how map works in python

# x = [1,2,3,4,5]

# def funcc(y):
#     return y - 1

# i = list(map(funcc, x))
# print(i)
# print(x)

In [71]:
# # using global var inside function

# x = 12
# def func():
#     global x
#     x +=3

# func()
# print(x)

In [72]:
# print("salam", end = " ")
# print("salam")
# print("salam")

In [73]:
# ll = [1,2,3,4,5]
# print([x - 1 for x in ll ])

In [74]:
# #queue in python:

# q = Queue()
# q.put(1)
# q.put(2)
# q.put(3)
# # print(1 in q)
# print(q.get())
# print(q.get())
# print(q.get())

In [75]:
#how to see if elem in list x:

# x = [1,2,3]
# print(12 in x)

In [76]:
#so None works!
# aa = None

# if aa == 12:
#     print("ok")
# else:
#     print("ok2")

In [77]:
aa = [1,2,3]
bb = [3,2,1]

Counter(aa) == Counter(bb)

print(aa)
print (bb)

# a = 67 

# def func():
#     print(a)

# # x = [12,13,True,"salam"]
# # if x == "salam":
# #     print("yes")
# # print("no")

# # print(range(10))
# func()

# d = {
#     1 : [1,2,3],
#     2 : [1,2,3],
#     9 : [1,2,3],
#     "salam" : 78,
#     4 : [1,2,3],
# }
# d["salam"] += 100
# print(d)

# s = "salam balajan"
# print(s[2:])
# sss = "33"
# b = int(sss.split(" "))
# # print(a)
# print(b)
# type(b)

[1, 2, 3]
[3, 2, 1]


In [79]:
# d = {
#     1 : [1,2,3],
#     2 : [1,2,3],
#     9 : [1,2,3],
#     4 : [1,2,3],
# }

# l = list(d.keys())
# print(l)
# print(type(l))
# x = [1,4,"salam", 7]
# print(dict.fromkeys(x, 0))
# len(d)

# for recipe in d.values():
#     print(recipe)
#     print(type(recipe))



In [ ]:
# How deep copy works in python:
# g = Graph(12)
# h = copy.deepcopy(g)
# print(g.v)
# h.v = 33
# print(g.v)